# Predict tags on StackOverflow with linear models

In this assignment you will learn how to predict tags for posts from [StackOverflow](https://stackoverflow.com). To solve this task you will use multilabel classification approach.

### Libraries

In this task you will need the following libraries:
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [Pandas](https://pandas.pydata.org) — a library providing high-performance, easy-to-use data structures and data analysis tools for the Python
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [NLTK](http://www.nltk.org) — a platform to work with natural language.

### Data

You can find all data required for this assignment into the folder `/data`.

### Text preprocessing

For this assignment you will need to use a list of stop words. It can be downloaded from *nltk*:

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddharthmehta/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In this task you will deal with a dataset of post titles from StackOverflow. You are provided a split to 3 sets: *train*, *validation* and *test*. All corpora (except for *test*) contain titles of the posts and corresponding tags (100 tags are available). The *test* set doesn't contain answers. Upload the corpora using *pandas* and look at the data:

In [2]:
from ast import literal_eval
import pandas as pd
import numpy as np

Literal_eval package takes care of the preprocessing for the string so that it can be used in python. To know more on literal_eval please see the below documentation <br>
https://kite.com/python/docs/ast.literal_eval

## Task 1: Create training, testing and validation data from the files given. Use title to be the independent variable and tags to be the dependent variable ( 5 points)

Note: Ensure you apply literal_eval function on the tags column to ensure all the tags are readable in python

In [33]:
train_data = pd.read_csv("train.tsv", header=0,delimiter="\t", quoting=3)
test_data = pd.read_csv("test.tsv", header=0,delimiter="\t", quoting=3)
validation_data = pd.read_csv("validation.tsv", header=0,delimiter="\t", quoting=3)

In [35]:
print(train_data.shape)
print(test_data.shape)
print(validation_data.shape)

(100000, 2)
(20000, 1)
(30000, 2)


In [28]:
train_data.head()

,title,tags
0,How to draw a stacked dotplot in R?,['r']
1,mysql select all records where a datetime fiel...,"['php', 'mysql']"
2,How to terminate windows phone 8.1 app,['c#']
3,get current time in a specific country via jquery,"['javascript', 'jquery']"
4,Configuring Tomcat to Use SSL,['java']


In [36]:
test_data.head()

,title
0,Warning: mysql_query() expects parameter 2 to ...
1,get click coordinates from <input type='image'...
2,How to implement cloud storage for media asset...
3,What is catcomplete in jQuery's autocomplete p...
4,Error building Android app with Cordova 3.1 CLI


In [37]:
validation_data.head()

,title,tags
0,Why odbc_exec always fail?,"['php', 'sql']"
1,Access a base classes variable from within a c...,['javascript']
2,"""Content-Type """"application/json"""" not require...","['ruby-on-rails', 'ruby']"
3,Sessions in Sinatra: Used to Pass Variable,"['ruby', 'session']"
4,"""Getting error - type """"json"""" does not exist ...","['ruby-on-rails', 'ruby', 'json']"


In [38]:
print(train_data.columns.values)
print(test_data.columns.values)
print(validation_data.columns.values)

['title' 'tags']
['title']
['title' 'tags']


In [39]:
print(train_data.title[0])
print(train_data.tags[0])

How to draw a stacked dotplot in R?
['r']


In [40]:
train_data["tags"] = train_data["tags"].apply(literal_eval)
validation_data["tags"] = validation_data["tags"].apply(literal_eval)

In [877]:
X_train = train_data.loc[:, train_data.columns == 'title']
X_val = validation_data.loc[:, validation_data.columns == 'title']
X_test = test_data.loc[:, test_data.columns == 'title']
Y_train = train_data.loc[:, train_data.columns == 'tags']
Y_val = validation_data.loc[:, validation_data.columns == 'tags']

In [878]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
title    100000 non-null object
dtypes: object(1)
memory usage: 781.3+ KB


In [879]:
Y_train.head()

,tags
0,[r]
1,"[php, mysql]"
2,[c#]
3,"[javascript, jquery]"
4,[java]


## Task 2 (Pre-processing). Implement the function *text_prepare* following the instructions. After that, run the function *test_test_prepare* to test it on tiny cases. (10 points)

One of the most known difficulties when working with natural data is that it's unstructured. For example, if you use it "as is" and extract tokens just by splitting the titles by whitespaces, you will see that there are many "weird" tokens like *3.5*, *?*,  *{}*, etc. To prevent the problems, it's usually useful to prepare the data in a custom way


In [918]:
import re

In [919]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text): ### The function will take in text and lower case it remove the stopwords, symbols and return it.
    text = text.lower()            ### Write a code which can change the input text to lowercase.
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text)             ### Write a code which replaces REPLACE_BY_SPACE_RE (above mentioned) symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text)             ### Write a code which deletes symbols which are in BAD_SYMBOLS_RE (above mentioned) from text
    text = [w for w in text.split(" ") if not w in STOPWORDS]  
    ### Write a code which deletes stopwords from text
    return " ".join(text)

def label_prepare(label):
    return " ".join(label)

In [920]:
def test_text_prepare():
    examples = ["SQL Server- any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int>* arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        if text_prepare(ex) != ans:
            print(text_prepare(ex))
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

Execute the test_text_prepare function

In [921]:
test_text_prepare()

'Basic tests are passed.'

*Note: You should pass the above test to ensure the text preprocessing is done before our analysis*

Now we can preprocess the titles using function *text_prepare* and  making sure that the headers don't have bad symbols:

In [923]:
X_train["title"] = X_train["title"].apply(text_prepare)
X_val["title"] = X_val["title"].apply(text_prepare)
X_test["title"] = X_test["title"].apply(text_prepare)

Y_train["tags"] = Y_train["tags"].apply(label_prepare)
Y_val["tags"] = Y_val["tags"].apply(label_prepare)

/Users/siddharthmehta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/siddharthmehta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/siddharthmehta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

Print the top 5 elements in x_train

In [924]:
print(X_train.shape)

(100000, 1)


In [925]:
print(Y_train.shape)

(100000, 1)


In [930]:
num_words = X["title"].size
num_tags = Y["tags"].size

def convertWordsToList(feature_data):
    clean_train_words = []
    num_words = feature_data["title"].size
    for i in range( 0, num_words):
        list_words = feature_data["title"][i].split(" ")
        for word in list_words:
            if(word == ''):
                continue
            clean_train_words.append(word)
    return clean_train_words

def convertTagsToList(label_data):  
    clean_train_tags = []
    num_tags = label_data["tags"].size
    for i in range( 0, num_tags):
        list_tags = label_data["tags"][i].split(" ")
        for tag in list_tags:
            clean_train_tags.append(tag)
    return clean_train_tags

In [931]:
X_train_clean = convertWordsToList(X_train)
X_val_clean = convertWordsToList(X_val)
X_test_clean = convertWordsToList(X_test)

In [932]:
Y_train_clean = convertTagsToList(Y_train)
Y_val_clean = convertTagsToList(Y_val)

## Task 2 (WordsTagsCount) - Find 3 most popular tags and 3 most popular words in the train data. - 5 points

Note: The words which appear the most are considered as popular in this case!

In [934]:
import collections
counter_words = collections.Counter(X_train_clean)

In [935]:
words_counts = dict(counter_words.items())
print("Most Common Words: \n", counter_words.most_common(3))

Most Common Words: 
 [('using', 8278), ('php', 5614), ('java', 5501)]


In [936]:
counter_tags = collections.Counter(Y_train_clean)

In [937]:
tags_counts = dict(counter_tags.items())
print("Most Common Tags: \n", counter_tags.most_common(3))

Most Common Tags: 
 [('javascript', 19078), ('c#', 19077), ('java', 18661)]


We are assuming that *tags_counts* and *words_counts* are dictionaries like `{'some_word_or_tag': frequency}`. After applying the sorting procedure, results will be look like this: `[('most_popular_word_or_tag', frequency), ('less_popular_word_or_tag', frequency), ...]`.

eg: 
Tag 1 - 100 Tag 2 - 65 Tag 3 - 250 <br>
after sorting looks like, <br>
Tag 3 - 250 Tag 1 - 100 Tag 2 - 65

## Task - 3 Transforming text to a vector (10 points)

Machine Learning algorithms work with numeric data and we cannot use the provided text data "as is". There are many ways to transform text data to numeric vectors. In this task you will try to use two of them.

#### Bag of words

One of the well-known approaches is a *bag-of-words* representation. To create this transformation, follow the steps:
1. Find *N* most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.
2. For each title in the corpora create a zero vector with the dimension equals to *N*.
3. For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.

Let's try to do it for a toy example. Imagine that we have *N* = 4 and the list of the most popular words is 

    ['hi', 'you', 'me', 'are']

Then we need to numerate them, for example, like this: 

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

And we have the text, which we want to transform to the vector:

    'hi how are you'

For this text we create a corresponding zero vector 

    [0, 0, 0, 0]
    
And iterate over all words, and if the word is in the dictionary, we increase the value of the corresponding position in the vector:

    'hi':  [1, 0, 0, 0]
    'how': [1, 0, 0, 0] # word 'how' is not in our dictionary
    'are': [1, 0, 0, 1]
    'you': [1, 1, 0, 1]

The resulting vector will be 

    [1, 1, 0, 1]
   
Implement the described encoding in the function *my_bag_of_words* with the size of the dictionary equals to 5000. To find the most common words use train data. You can test your code using the function *test_my_bag_of_words*.

In [938]:
DICT_SIZE = 5000
words_counts = dict(counter_words.most_common(DICT_SIZE))
INDEX_TO_WORDS = sorted(words_counts.keys(), key = lambda x:words_counts[x], reverse = True)
WORDS_TO_INDEX = {word:i for i,word in enumerate(INDEX_TO_WORDS)}

def my_bag_of_words(text, words_to_index, dict_size):
    result_vector = np.zeros(dict_size)
    for ex in zip(text):
        for word in ex:
            if word in words_to_index:
                result_vector[words_to_index[word]] += 1
        return result_vector

In [939]:
for ex in zip(X_train["title"]):
    print(ex)
    break

('draw stacked dotplot r',)


In [940]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

Execute the test_text_prepare function <br>
*<u>Note:</u> You should pass the above test to ensure BOW is working correctly!*

In [941]:
#Kindly Ignore

Now apply the implemented function to all samples (this might take up to a minute):

In [942]:
import scipy
from scipy import sparse as sp_sparse

In [943]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(X_train["title"], WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(X_val["title"], WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(X_test["title"], WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (1, 5000)
X_val shape  (1, 5000)
X_test shape  (1, 5000)


As you might notice, we transform the data to sparse representation, to store the useful information efficiently. There are many types: of such representations, however sklearn algorithms can work only with  csr matrix, so we will use this one.<br>
<u>Documentations on sparse matrix:</u> <br>
(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) 
(https://docs.scipy.org/doc/scipy/reference/sparse.html)



In [944]:
sparse = X_train_mybag.transpose()
non_zero_elements_count = sparse[11].count_nonzero()
print(non_zero_elements_count)

0


## Task 4 - TF-IDF (5 points)

The second approach extends the bag-of-words framework by taking into account total frequencies of words in the corpora. It helps to penalize too frequent words and provide better features space. 

Implement function *tfidf_features* using class  from *scikit-learn*. Use *train* corpus to train a vectorizer. Don't forget to take a look into the arguments that you can pass to it. We suggest that you filter out too rare words (occur less than in 5 titles) and too frequent words (occur more than in 90% of the titles). Also, use bigrams along with unigrams in your vocabulary. 

## Write a function which takes x_train, x_val and x_test as input and return the tf-idf features of the same and the vocabulary

In [945]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf_features(X_train, X_val, X_test):
    tfidf_vectorizer = TfidfVectorizer(max_df = 0.90, min_df = 5, ngram_range = (1,2))
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.fit_transform(X_test)
    X_val_tfidf = tfidf_vectorizer.fit_transform(X_val)
    return X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_

In [946]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train["title"].tolist(), X_val["title"].tolist(), X_test["title"].tolist())
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

Print the index of string "C#" in the vocabulary

In [947]:
count = 0
for key in tfidf_vocab.keys():
    if (tfidf_vocab.get(key) == "C#"):
        print(key)
        count = count + 1
if(count == 0):
    print("Given Key not Present")

Given Key not Present


## Task 5: Classification (15 points)
MultiLabel classifier

As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose it is convenient to use MultiLabelBinarizer from sklearn. <br>
<u>Documentation:</u> <br>
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

In [948]:
from sklearn.preprocessing import MultiLabelBinarizer

In [949]:
mlb = MultiLabelBinarizer(classes=sorted(counter_tags.keys()))
y_train = mlb.fit_transform(Y_train)
y_val = mlb.fit_transform(Y_val)

/Users/siddharthmehta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['a', 'g', 's', 't'] will be ignored
  .format(sorted(unknown, key=str)))


In this task we suggest to use One-vs-Rest approach, which is implemented in [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) class. In this approach *k* classifiers (= number of tags) are trained. As a basic classifier, use [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time, because a number of classifiers to train is large.

**OneVsRest multi-label strategy**

The Multi-label algorithm accepts a binary mask over multiple labels. The result for each prediction will be an array of 0s and 1s marking which class labels apply to each row input sample.

**Logistic Regression & SVM**

OneVsRest strategy can be used for multi-label learning, where a classifier is used to predict multiple labels for instance. LR & SVM supports multi-class, but we are in a multi-label scenario, therefore, we wrap classifiers in the OneVsRestClassifier.

*If you want to learn more about OneVsRest, check out these links:*
- *https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5*
- *https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff*
- *https://medium.com/coinmonks/multi-label-classification-blog-tags-prediction-using-nlp-b0b5ee6686fc*

In [950]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline

In [951]:
def train_classifier(x_tr, y_tr):
    # Create and fit LogisticRegression and LinearSVC wraped into OneVsRestClassifier.
    clf = OneVsRestClassifier(LogisticRegression(solver = 'sag'))
    clf.fit(x_tr, y_tr)
    return clf  ### clf is the model

Train the classifiers for different data transformations: *bag-of-words* and *tf-idf*.
classifier_mybag = model for 

In [952]:
sorted(counter_tags.keys())

['.net',
 'ajax',
 'algorithm',
 'android',
 'angularjs',
 'apache',
 'arrays',
 'asp.net',
 'asp.net-mvc',
 'c',
 'c#',
 'c++',
 'class',
 'cocoa-touch',
 'codeigniter',
 'css',
 'csv',
 'database',
 'date',
 'datetime',
 'django',
 'dom',
 'eclipse',
 'entity-framework',
 'excel',
 'facebook',
 'file',
 'forms',
 'function',
 'generics',
 'google-maps',
 'hibernate',
 'html',
 'html5',
 'image',
 'ios',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'json',
 'jsp',
 'laravel',
 'linq',
 'linux',
 'list',
 'loops',
 'maven',
 'mongodb',
 'multithreading',
 'mysql',
 'node.js',
 'numpy',
 'objective-c',
 'oop',
 'opencv',
 'osx',
 'pandas',
 'parsing',
 'performance',
 'php',
 'pointers',
 'python',
 'python-2.7',
 'python-3.x',
 'qt',
 'r',
 'regex',
 'rest',
 'ruby',
 'ruby-on-rails',
 'ruby-on-rails-3',
 'selenium',
 'servlets',
 'session',
 'sockets',
 'sorting',
 'spring',
 'spring-mvc',
 'sql',
 'sql-server',
 'string',
 'swift',
 'swing',
 'twitter-bootstrap',
 'uitableview',
 '

Now you can create predictions for the data. You will need two types of predictions: labels and scores.

In [953]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

y_val_predicted_labels_mybag

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Now, we would need to compare the results of different predictions, e.g. to see whether TF-IDF transformation helps or to try different regularization techniques in logistic regression. For all these experiments, we need to setup evaluation procedure. 

## Evaluation (10 points)

To evaluate the results we will use several classification metrics:
 - [Accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
 - [F1-score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)
 - [Area under ROC-curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
 - [Area under precision-recall curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score) 
 
Make sure you are familiar with all of them. If you want a refresher, you can click the link to their documentation

## Import the necessary libraries for the above metrics

In [954]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

Define the function *print_evaluation_scores* which takes y_val and predicted as input calculates and prints the following output:
 - *accuracy*
 - *F1-score - Average = 'weighted'* 
 - *Precision - Average = 'macro'*

In [956]:
def print_evaluation_scores(y_val, predicted):
    print(accuracy_score(y_val, predicted))
    print(f1_score(y_val, predicted))

In [961]:
print('Bag-of-words')
#print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

Bag-of-words
Tfidf


ValueError: Found input variables with inconsistent numbers of samples: [1, 30000]

You might also want to plot some form of the [ROC curve](http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) for the case of multi-label classification. The input parameters for the roc curve are:
 - true labels
 - decision functions scores
 - number of classes

Import the roc_auc function from the metrics.py file provided

## Task 4 (MultilabelClassification) - Optional 
** Once we have the evaluation set up, we suggest that you experiment a bit with training your classifiers. We will use *F1-score weighted* as an evaluation metric. Our recommendation:
- compare the quality of the bag-of-words and TF-IDF approaches and chose one of them.
- for the chosen one, try *L1* and *L2*-regularization techniques in Logistic Regression with different coefficients (e.g. C equal to 0.1, 1, 10, 100).



In [0]:
######################################
######### YOUR CODE HERE #############
######################################

When you are happy with the quality, create predictions for *test* set.